In [ ]:
import xarray as xr
import pandas as pd
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import numpy as np
import os
import re

# Primary exploration: reproducibility.

The idea behind this notebook is to study the data and compare what we get by calculating things and the corresponding values stored in the repo.

Let's start by picking one of the TARA matrices stored in the repo.

In [ ]:
df1 = pd.read_csv('../01_data/02_satellite_data_processed/matrix_tara_world_adj_grids_25.tsv', sep='\t', index_col=0)
if 'IOP.aph_44' in df1.columns and 'bbp_unc_443' in df1.columns:
    df1 = df1.drop(columns = ['IOP.aph_44','bbp_unc_443'])

In [ ]:
df1

Now we define it's 'new' counterpart, obtained in 28/10 via the `00_00_extract_satellite_data.py` script. 

In [ ]:
df2 = pd.read_csv('../01_data/02_satellite_data_processed/matrix_tara_world_adj_grids_25_new.tsv', sep='\t', index_col=0)
if 'IOP.aph_44' in df2.columns and 'bbp_unc_443' in df2.columns:
    df2= df2.drop(columns = ['IOP.aph_44','bbp_unc_443'])

We start by checking the shape of the DataFrames match.

In [ ]:
(df1.columns == df2.columns).all()

Now we check the values are the same.

In [ ]:
diff = (df1.values - df2.values)
total_err = diff.sum()
print(total_err/df1.values.shape[0]*df1.values.shape[1])

This is way too high, so lets dig into that error.

In [ ]:
idxs = diff.nonzero()
for k in range(len(idxs[0])):
    i = idxs[0][k]
    j = idxs[1][k]
    print(f" Difference at ({i},{j}) = {diff[i,j]}")

So there is only one 'big' error in the last element, meanwhile the rest can be attributed to float point. The numerical coordinates of the attribute are (79,31) as we can see above.

In [ ]:
df1.iloc[79].keys()[31]

So the problematic sample is `TSC280`, and the problematic feature is `PAR.par`. Looking at the individual values:

In [ ]:
print(f"Repo-stored value: {df1['PAR.par'].loc['TSC280']}.\nNew obtained value: {df2['PAR.par'].loc['TSC280']}.")

Let's try to recalculate this value 'by hand' and compare it to the two that we have. This will be done with the source code of the project.

In [ ]:
import numpy as np
import os
import re

def find_satellite_file(directory, pattern):
    regex = re.compile(pattern)
    for file in os.listdir(directory):
        if regex.match(file):
            return os.path.join(directory, file)
    return None

def select_n_nearest_valid(ds, feature, latitude, longitude, n):
    latitudes = ds['lat'].values
    longitudes = ds['lon'].values
    
    lat_grid, lon_grid = np.meshgrid(latitudes, longitudes, indexing='ij')
    distances = np.sqrt((lat_grid - latitude)**2 + (lon_grid - longitude)**2)
    
    sorted_indices = np.unravel_index(np.argsort(distances, axis=None), distances.shape)
    
    valid_points = []
    for i in range(len(sorted_indices[0])):
        lat_idx = sorted_indices[0][i]
        lon_idx = sorted_indices[1][i]
        data_point = ds[feature].isel(lat=lat_idx, lon=lon_idx)
        if not np.isnan(data_point.values):
            valid_points.append(data_point.values)
        if len(valid_points) >= n:
            break
    
    if len(valid_points) > 0:
        return np.mean(valid_points)
    else:
        return np.nan
n = 25 #number of near points.
file_path = '../01_data/01_biological_data'
file_name = 'metadata.tsv'
sat_data_path = '../01_data/00_satellite_data'

output_dir = '../01_data/02_satellite_data_processed'
os.makedirs(output_dir, exist_ok=True)

file = os.path.join(file_path, file_name)
md = pd.read_csv(file, sep='\t', index_col=0)

md_srf = md[md.Layer == 'SRF'].copy()
md_srf['Event.date.YYYYMM'] = md_srf['Event.date'].str[:7].str.replace('-', '')
md_srf['Event.date.YYYYMM01'] = md_srf['Event.date'].str[:7].str.replace('-', '')+'01'

satellite_features = [
    'CHL.chlor_a', 'FLH.nflh', 'FLH.ipar', 'IOP.adg_unc_443', 'IOP.adg_443',
    'IOP.aph_unc_443', 'IOP.aph_44', 'IOP.bbp_s', 'IOP.adg_s', 'bbp_unc_443', 
    'IOP.bbp_443', 'IOP.a_412', 'IOP.a_443', 'IOP.a_469', 'IOP.a_488', 'IOP.a_531', 
    'IOP.a_547', 'IOP.a_555', 'IOP.a_645', 'IOP.a_667', 'IOP.a_678', 'IOP.bb_412', 
    'IOP.bb_443', 'IOP.bb_469', 'IOP.bb_488', 'IOP.bb_531', 'IOP.bb_547', 'IOP.bb_555', 
    'IOP.bb_645', 'IOP.bb_667', 'IOP.bb_678', 'KD.Kd_490', 'NSST.sst', 'PAR.par', 
    'PIC.pic', 'POC.poc', 'RRS.aot_869', 'RRS.angstrom', 'RRS.Rrs_412', 'RRS.Rrs_443', 
    'RRS.Rrs_469', 'RRS.Rrs_488', 'RRS.Rrs_531', 'RRS.Rrs_547', 'RRS.Rrs_555', 
    'RRS.Rrs_645', 'RRS.Rrs_667', 'RRS.Rrs_678', 'SST.sst'
]

satellite_data_terra = pd.DataFrame(index=md_srf.index, columns=satellite_features)
satellite_data_aqua = pd.DataFrame(index=md_srf.index, columns=satellite_features)
row = md_srf.loc['TSC280'] #We use the obtained key.
feature = 'PAR.par' #We use the obtained feature.
latitude = row['Latitude']
longitude = row['Longitude']
date = row['Event.date.YYYYMM']
resolution = '9km'
pattern_terra = rf"TERRA_MODIS\.{date}01_{date}\d{{2}}\.L3m\.MO\.{feature}\.{resolution}\.nc"
file_path_terra = find_satellite_file(sat_data_path, pattern_terra)
pattern_aqua = rf"AQUA_MODIS\.{date}01_{date}\d{{2}}\.L3m\.MO\.{feature}\.{resolution}\.nc"
file_path_aqua = find_satellite_file(sat_data_path, pattern_aqua)
ds_terra = xr.open_dataset(file_path_terra)
ds_aqua = xr.open_dataset(file_path_aqua)
terra_values = []
aqua_values = []
try:
    variable_name = feature.split('.')[1]
    
    data_point_terra = select_n_nearest_valid(ds_terra, variable_name, latitude, longitude, 25)
    terra_values.append(data_point_terra)

    data_point_aqua = select_n_nearest_valid(ds_aqua, variable_name, latitude, longitude, 25)
    aqua_values.append(data_point_aqua)

except KeyError:
    print(f"Feature {feature} not found in dataset")
finally:
    ds_terra.close()
    ds_aqua.close()
aqua_arr = np.array(aqua_values)
terra_arr = np.array(terra_values)
final_arr = (aqua_arr + terra_arr)*0.5
print(f'Value by hand: {final_arr[0]}.\nValue stored in repo: {df1['PAR.par'].loc['TSC280']}. \nNew value obtained by script: {df2['PAR.par'].loc['TSC280']}.')

This means the value obtained by hand is closer to the '_new' value than to the one stored in the repo, with error likely of approximation. The difference of the old value and the new value is of order $10^{-3}$.

Now let us make a simmilar study to the `n=1` case: 

In [ ]:
df3 = pd.read_csv("../01_data/02_satellite_data_processed/matrix_tara_world_adj_grids_01.tsv", sep='\t', index_col=0)
if 'IOP.aph_44' in df3.columns and 'bbp_unc_443' in df3.columns:
    df3 = df3.drop(columns = ['IOP.aph_44','bbp_unc_443'])

Now we define it's 'new' counterpart, obtained in 28/10 via the `00_00_extract_satellite_data.py` script. 

In [ ]:
df4 = pd.read_csv('../01_data/02_satellite_data_processed/matrix_tara_world_adj_grids_01_new.tsv', sep='\t', index_col=0)
if 'IOP.aph_44' in df4.columns and 'bbp_unc_443' in df4.columns:
    df4= df4.drop(columns = ['IOP.aph_44','bbp_unc_443'])

We start by checking the shape of the DataFrames match.

In [ ]:
(df3.columns == df4.columns).all()

Now we check the values are the same.

In [ ]:
diff2 = (df3.values - df4.values)
print(diff2.max())
print(diff2.min())

In [ ]:
total_err2 = diff2.sum()
print(total_err2)

In this case, the values are all equal.

This gives the impression that the issue is somewhere between the part where the closest sat reads (physicall distance-wise) are chosen. There is a pending plan of testing with a third run. 

To investigate further one may plot the points selected in the proccess for both versions and check if they are the same or not.

It might also be related to eventual changes to the original database.